In [6]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor

In [7]:
print(torch.__version__)

2.6.0+cpu


In [8]:
train_dt = datasets.MNIST(root='./data', train=True, transform=ToTensor(), download=True)

100.0%
100.0%
100.0%
100.0%


In [11]:
test_dt = datasets.MNIST(root='./data', train=False, transform=ToTensor(), download=True)

In [12]:
train_dt

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [13]:
test_dt

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()